<a href="https://colab.research.google.com/github/SUNANUS/codezip/blob/main/Decision_Tree__First_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
train_df = pd.read_csv(train_path, sep=",")
test_df = pd.read_csv(test_path, sep=",")

In [ ]:
train_df.info()

In [ ]:
#전처리-제거
train_df = train_df.drop(['PassengerId', 'Name','Cabin'], axis=1)
test_df = test_df.drop(['Name','Cabin'], axis=1)

In [ ]:
#전처리 -결측치 채워넣기
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())

most_freq = train_df['HomePlanet'].value_counts(dropna=True).idxmax()
train_df['HomePlanet'].fillna(most_freq, inplace = True)

most_freq = train_df['CryoSleep'].value_counts(dropna=True).idxmax()
train_df['CryoSleep'].fillna(most_freq, inplace = True)

most_freq = train_df['Destination'].value_counts(dropna=True).idxmax()
train_df['Destination'].fillna(most_freq, inplace = True)

most_freq = train_df['VIP'].value_counts(dropna=True).idxmax()
train_df['VIP'].fillna(most_freq, inplace = True)

#test 데이터도 전처리해주기
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())

most_freq = test_df['HomePlanet'].value_counts(dropna=True).idxmax()
test_df['HomePlanet'].fillna(most_freq, inplace = True)

most_freq = test_df['CryoSleep'].value_counts(dropna=True).idxmax()
test_df['CryoSleep'].fillna(most_freq, inplace = True)

most_freq = test_df['Destination'].value_counts(dropna=True).idxmax()
test_df['Destination'].fillna(most_freq, inplace = True)

most_freq = test_df['VIP'].value_counts(dropna=True).idxmax()
test_df['VIP'].fillna(most_freq, inplace = True)

In [ ]:
#전처리 -새로운 열추가
train_df['Paid service']= 0
train_df['Paid service'] = np.where((train_df['RoomService']>0)|(train_df['FoodCourt']>0)|(train_df['ShoppingMall']>0)|(train_df['Spa']>0)|(train_df['VRDeck']>0), 1, 0)

#test 데이터도 전처리해주기
test_df['Paid service']= 0
test_df['Paid service'] = np.where((test_df['RoomService']>0)|(test_df['FoodCourt']>0)|(test_df['ShoppingMall']>0)|(test_df['Spa']>0)|(test_df['VRDeck']>0),1, 0)

In [ ]:
#전처리 - One-hot Encoding
train_df["HomePlanet_Earth"]=train_df["HomePlanet"] =="Earth"
train_df["HomePlanet_Europa"]=train_df["HomePlanet"] =="Europa"
train_df["HomePlanet_Mars"]=train_df["HomePlanet"] =="Mars"

train_df["Destination_T"]=train_df["Destination"] =="TRAPPIST-1e"
train_df["Destination_55"]=train_df["Destination"] =="55 Cancri e"
train_df["Destination_P"]=train_df["Destination"] =="PSO J318.5-22"

#test 데이터도 전처리해주기
test_df["HomePlanet_Earth"]=test_df["HomePlanet"] =="Earth"
test_df["HomePlanet_Europa"]=test_df["HomePlanet"] =="Europa"
test_df["HomePlanet_Mars"]=test_df["HomePlanet"] =="Mars"

test_df["Destination_T"]=test_df["Destination"] =="TRAPPIST-1e"
test_df["Destination_55"]=test_df["Destination"] =="55 Cancri e"
test_df["Destination_P"]=test_df["Destination"] =="PSO J318.5-22"

train_df.head(10)

In [ ]:
feature_names = ["HomePlanet_Earth", "HomePlanet_Europa", "HomePlanet_Mars","CryoSleep",
                 "Destination_T", "Destination_55", "Destination_P", 
                 "Age","VIP","Paid service"]
                 

X_train = train_df[feature_names]

print(X_train.shape)
X_train.head()

In [ ]:
label_name = "Transported"

Y_train = train_df[label_name]

print(Y_train.shape)

Y_train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier()

In [ ]:
#최적하이퍼파라미터찾기
params = {
    'max_depth': [4, 5, 6, 7, 8],
}

grid_dclf = GridSearchCV(model, param_grid=params, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, Y_train)

In [ ]:
#후보 중 최적 하이퍼 파라미터 = 5 확인
print('최적 하이퍼 파라미터:', grid_dclf.best_params_)

In [ ]:
model = DecisionTreeClassifier(max_depth= 5)

model.fit(X_train, Y_train)

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

export_graphviz(model,
                feature_names=feature_names,
                class_names=["False", "True"],
                out_file="decision-tree.dot")

with open("decision-tree.dot") as f:
    dot_graph = f.read()
    
graphviz.Source(dot_graph)

In [ ]:
X_test=test_df[feature_names]
model.predict(X_test)

In [ ]:
prediction_list=model.predict(X_test)

result =pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': prediction_list
})
result.to_csv('./suna.csv', index=False) 